In [20]:
# import libraries
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import datetime
from sklearn import preprocessing, linear_model, model_selection

%matplotlib inline

Класс моделей ARIMA недостаточно богат для наших данных: с их помощью, например, никак нельзя учесть взаимосвязи между рядами. Это можно сделать с помощью векторной авторегрессии VARIMA, но её питоновская реализация не позволяет использовать регрессионные признаки. Кроме того, авторегрессионный подход не позволяет учитывать, например, взаимодействия между сезонными компонентами. Вы могли заметить, что форма суточных сезонных профилей в будни и выходные немного разная; явно моделировать этот эффект с помощью ARIMA не получится.

Нам нужна более сложная модель. Давайте займёмся сведением задачи массового прогнозирования рядов к регрессионной постановке! Вам понадобится много признаков. Некоторые из них у вас уже есть — это:
<ol>
<li>идентификатор географической зоны
<li>дата и время
<li>количество поездок в периоды, предшествующие прогнозируемому
<li>синусы, косинусы и тренды, которые вы использовали внутри регрессионной компоненты ARIMA
<li>Кроме того, не спешите выбрасывать построенный вами на прошлой неделе прогнозы — из них может получиться хороший признак для регрессии!
</ol>



Вы можете попробовать разные регрессионный модели, но хорошие результаты, скорее всего, дадут такие, которые будут позволять признакам взаимодействовать друг с другом.

Поскольку прогноз нужен на 6 часов вперёд, проще всего будет построить 6 независимых регрессионных моделей — одна для прогнозирования y^T+1|T, другая для y^T+2|T и т.д.

<ol>Чтобы сдать задание, выполните следующую последовательность действий.
<li>Для каждой из шести задач прогнозирования y^T+i|T,i=1,…,6 сформируйте выборки. Откликом будет yT+i при всевозможных значениях T, а признаки можно использовать следующие:
<ul>
<li>идентификатор географической зоны — категориальный
<li>год, месяц, день месяца, день недели, час — эти признаки можно пробовать брать и категориальными, и непрерывными, можно даже и так, и так (done)
<li>синусы, косинусы и тренды, которые вы использовали внутри регрессионной компоненты ARIMA (done)
<li>сами значения прогнозов ARIMA y^T+i|TARIMA
<li>количество поездок из рассматриваемого района в моменты времени yT,yT−1,…,yT−K (параметр K можно подбирать; попробуйте начать, например, с 6)
<li>количество поездок из рассматриваемого района в моменты времени yT−24,yT−48,…,yT−24∗Kd (параметр Kd можно подбирать; попробуйте начать, например, с 2)
<li>суммарное количество поездок из рассматриваемого района за предшествующие полдня, сутки, неделю, месяц
</ul>
Будьте внимательны при создании признаков — все факторы должны быть рассчитаны без использования информации из будущего: при прогнозировании y^T+i|T,i=1,…,6 вы можете учитывать только значения y до момента времени T включительно.


<li>Выбранными моделями постройте для каждой географической зоны и каждого конца истории от 2016.04.30 23:00 до 2016.05.31 17:00 прогнозы на 6 часов вперёд; посчитайте в ноутбуке ошибку прогноза по следующему функционалу:
Qmay=1R∗739∗6∑r=1R∑T=2016.04.3023:002016.05.3117:00∑i=16y^T|T+ir−yT+ir.
Убедитесь, что ошибка полученных прогнозов, рассчитанная согласно функционалу Q, определённому на прошлой неделе, уменьшилась по сравнению с той, которую вы получили методом индивидуального применения моделей ARIMA. Если этого не произошло, попробуйте улучшить ваши модели.

<li>Итоговыми моделями постройте прогнозы для каждого конца истории от 2016.05.31 23:00 до 2016.06.30 17:00 и запишите все результаты в один файл в формате geoID, histEndDay, histEndHour, step, y. Здесь geoID — идентификатор зоны, histEndDay — день конца истории в формате id,y, где столбец id состоит из склеенных через подчёркивание идентификатора географической зоны, даты конца истории, часа конца истории и номера отсчёта, на который делается предсказание (1-6); столбец y — ваш прогноз.

<li>Загрузите полученный файл на kaggle: https://inclass.kaggle.com/c/yellowtaxi. Добавьте в ноутбук ссылку на сабмишн.

<li>Загрузите ноутбук в форму.

Подгружаем данные

In [21]:
# id нужных регионов
regsDf = pd.read_csv('../crowdRegs.csv',names=['id','regId']);  

# времянные ряды для этих регионов
df = pd.read_pickle('../loadData/crowdRegs3.pcl')
regNames = regsDf.regId.values.astype('str')
df.columns = regNames

Наверное, оптимальный способ - пройтись по всем регионам, сформировать требуемую выборку, а потом - состыковать. 
Вероятно, в процессе работы получится векторизовать это действие.
Пожалуй, имеет смысл сначала для всего фрейма добавить общие для всех колонок признаки (тренд, гармоники, даты, дни недели)

In [22]:
def processDataFrame(inpDf, Kw = 5, Ka = 2):
    """
    Обрабатываем сразу весь dateFrame и добавляем признаки, общие для всех рядов
    тренд, гармоники, категориальные перемнные
    для дат, дней недели, etc)

    Parameters:
    Kw number of weeks harmonics
    Ka number of annual harmonics
    """

    inpDf = inpDf.assign(linear = (inpDf.index - datetime.datetime(2014,1,1,0,0,0))/np.timedelta64(1, 'h'))
    
    # час — эти признаки можно пробовать брать и категориальными
    # и непрерывными, можно даже и так, и так

    # добавляем гармонические фичи
    for ind in range(1,Kw+1):
        inpDf['weekCos'+str(ind)]= np.cos(np.pi*inpDf.linear*ind/168)
        inpDf['weekSin'+str(ind)]= np.sin(np.pi*inpDf.linear*ind/168)
     
    for ind in range(1,Ka+1):
        inpDf['yearCos'+str(ind)]= np.cos(2*np.pi*inpDf.linear*ind/8766)        
        inpDf['yearSin'+str(ind)]= np.sin(2*np.pi*inpDf.linear*ind/8766)

    # добавляем числовое и категориальные свойства для дней недели
    inpDf = inpDf.assign(dayOfWeek = inpDf.index.dayofweek)
    lbDays = preprocessing.LabelBinarizer()
    lbDays.fit(list(np.arange(6)))
    DoW = pd.DataFrame(lbDays.transform(inpDf.index.dayofweek),columns = ['dayOfWeek_'+str(x) for x in np.arange(6)],
                       index = inpDf.index)      
    inpDf = inpDf.merge(DoW,left_index=True,right_index=True)

    # добавляем dummy variables для месяца
    inpDf = inpDf.assign(month = inpDf.index.month)
    lbMonths = preprocessing.LabelBinarizer()
    lbMonths.fit(list(np.arange(12)))
    Months = pd.DataFrame(lbMonths.transform(inpDf.index.month),columns = ['month_'+str(x) for x in np.arange(1,13)],
                          index = inpDf.index)      
    inpDf = inpDf.merge(Months,left_index=True,right_index=True);

    # добавляем год (вещественный)
    inpDf = inpDf.assign(year = inpDf.index.year)

    # добавляем день месяца (вещественный)
    inpDf = inpDf.assign(day = inpDf.index.day)

    # добавляем час (вещественный и категориальный)
    inpDf = inpDf.assign(hour = inpDf.index.hour)
    lbHours = preprocessing.LabelBinarizer()
    lbHours.fit(list(np.arange(24)))
    Hours = pd.DataFrame(lbHours.transform(inpDf.index.hour),columns = ['hour_'+str(x) for x in np.arange(24)],
                       index = inpDf.index)      
    inpDf = inpDf.merge(Hours,left_index=True,right_index=True)
    
    return inpDf

Теперь делаем индивидуальную обработку для каждого региона
<ol>
<li> добавляем идентификатор географической зоны — категориальный
<li> количество поездок из рассматриваемого района в моменты времени yT,yT−1,…,yT−K (параметр K можно подбирать; попробуйте начать, например, с 6)
<li> количество поездок из рассматриваемого района в моменты времени yT−24,yT−48,…,yT−24∗Kd (параметр Kd можно подбирать; попробуйте начать, например, с 2)
<li>суммарное количество поездок из рассматриваемого района за предшествующие полдня, сутки, неделю, месяц 
2) 
</ol>

In [23]:
def processSeries(df,tReg,Kh = 6, Kp = 2):
    """
    Обработка одного данного ряда 
    parameters:
        df - начальный датафрейм, из которого выберем для обработки один ряд
        tReg - название ряда, который надо обработать
        Kh - количество отслеживаемых прошлых суточных лагов "назад"
        Kp - количество отслеживаемых прошлых периодических лагов (период 24 часа)

    """
 

    tDf = df.loc[:,tReg.split() + commonFeatures].rename(columns={tReg:'y'})

    tDf = tDf.assign(region = tReg)

    for timeLag in np.arange(1,Kh+1):
        name = 'hourLag_'+str(timeLag)
        tDf.loc[:,name] = tDf.y.shift(periods=timeLag)

    for timeLag in np.arange(1,Kp+1):
        name = 'periodicLag_'+str(timeLag)
        tDf.loc[:,name] = tDf.y.shift(periods=timeLag*24)

    tDf.fillna(0,inplace=True)    

    # суммарное количество поездок из рассматриваемого района за предшествующие полдня, сутки, неделю, месяц
    tDf.loc[:,'sum12'] = tDf.y.rolling(window = 12, min_periods = 1).sum()
    tDf.loc[:,'sum24'] = tDf.y.rolling(window = 24, min_periods = 1).sum()
    tDf.loc[:,'sumWeek'] = tDf.y.rolling(window = 168, min_periods = 1).sum()
    tDf.loc[:,'sumMonth'] = tDf.y.rolling(window = 720, min_periods = 1).sum()
    
    #создаём шесть целевые переменных для каждого конца истории
    for targetVar in np.arange(1,7):
        name = 'y'+str(targetVar)
        tDf.loc[:,name] = tDf.y.shift(-targetVar)
    tDf.fillna(0,inplace=True)
    
    return tDf


In [24]:
# общая обработка данных
df2 = processDataFrame(df,Kw = 7, Ka = 5)
commonFeatures =  list(set(df2.columns)-set(df.columns.values))
# обработка отдельный рядов
df3 = pd.DataFrame()
for regName in regNames:
    df3 = pd.concat([df3, processSeries(df2,regName,Kh = 12, Kp = 4)])

In [25]:
regDf = df3.get('region')
regDf = regDf.reset_index()
regDf.rename(columns={'index':'date'},inplace=True)

Разбейте каждую из шести выборок на три части:
<ul>
<li> Обучающая, на которой будут настраиваться параметры моделей — всё до апреля 2016
<li> Тестовая, на которой вы будете подбирать значения гиперпараметров — май 2016
<li> Итоговая, которая не будет использоваться при настройке моделей вообще — июнь 2016
</ul>

In [26]:
df3.reset_index(inplace=True)
df3 = pd.get_dummies(df3,'region')

In [27]:
startTrain = '2014-01-01 00:00:00'
endTrain   = '2016-04-30 23:00:00'

startValidation = '2016-05-01 00:00:00'
endValidation   = '2016-05-31 23:00:00'

startTest = '2016-06-01 00:00:00'
endTest   = '2016-06-30 23:00:00'

In [28]:
trainSet = df3.query('index >= @startTrain and index <= @endTrain') 
validationSet = df3.query('index >= @startValidation and index <= @endValidation')
testSet = df3.query('index >= @startTest and index <= @endTest')

Выберите вашу любимую регрессионную модель и настройте её на каждом из шести наборов данных, подбирая гиперпараметры на мае 2016. Желательно, чтобы модель:
<ul>
<li>Допускала попарные взаимодействия между признаками
<li>Была устойчивой к избыточному количеству признаков (например, использовала регуляризаторы)
</ul>

In [29]:
testSet.head()

,index,y,weekSin1,weekSin2,month,hour_19,hour_9,hour_8,year,hour_5,...,region_1630,region_1684,region_1733,region_1734,region_1783,region_2068,region_2069,region_2118,region_2119,region_2168
21168,2016-06-01 00:00:00,26,-5.095769e-14,-1.019154e-13,6,0,0,0,2016,0,...,0,0,0,0,0,0,0,0,0,0
21169,2016-06-01 01:00:00,14,1.869887e-02,3.739119e-02,6,0,0,0,2016,0,...,0,0,0,0,0,0,0,0,0,0
21170,2016-06-01 02:00:00,5,3.739119e-02,7.473009e-02,6,0,0,0,2016,0,...,0,0,0,0,0,0,0,0,0,0
21171,2016-06-01 03:00:00,2,5.607045e-02,1.119645e-01,6,0,0,0,2016,0,...,0,0,0,0,0,0,0,0,0,0
21172,2016-06-01 04:00:00,1,7.473009e-02,1.490423e-01,6,0,0,0,2016,0,...,0,0,0,0,0,0,0,0,0,0


Попробуем обучить Ridge классификатор с L2 регуляризацией.

In [49]:
targetList = ['y1','y2','y3','y4','y5','y6']
paramsAlpha = np.linspace(0.2,2,5)

In [50]:
bestModels = dict()
for target in targetList:
    print target
    dropCols = [x for x in targetList if x != target ]
    dropCols.append('y')
    dropCols.append('index')
    tmpTrain = trainSet.drop(dropCols,axis = 1)
    tmpTrain.rename(columns={target:'y'},inplace=True)
    
    tmpValidation = validationSet.drop(dropCols,axis = 1)
    tmpValidation.rename(columns={target:'y'},inplace=True)
    
    minErr = np.inf
    bestAlpha = 0
    
    
    for a in paramsAlpha:
        regressor = linear_model.ElasticNet(alpha= a)
        regressor.fit(tmpTrain.drop('y',axis = 1),tmpTrain.loc[:,'y'])
        prediction = regressor.predict(tmpValidation.drop('y',axis = 1))
        err = np.abs(prediction - tmpValidation.loc[:,'y'])
        err = err.mean()
        print a, err
        if err <minErr:
            minErr = err
            bestAlpha = a
            bestModel = regressor
    
    bestModels.update({target:bestModel})
            
    print 'Smallest error {:2.3f} at a = {:2.3f}'.format(err,a)     

y1


C:\Anaconda2\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


0.2 30.3537252324
0.65 30.3925861949
1.1 30.4183938719
1.55 30.4243368721
2.0 30.4198267144
Smallest error 30.420 at a = 2.000
y2
0.2 41.9685838924
0.65 42.091605839
1.1 42.1419190039
1.55 42.1518007526
2.0 42.1433554047
Smallest error 42.143 at a = 2.000
y3
0.2 47.0919063516
0.65 47.1599091756
1.1 47.1516918978
1.55 47.1274092037
2.0 47.0991704764
Smallest error 47.099 at a = 2.000
y4
0.2 47.4079815039
0.65 47.3303732518
1.1 47.2744314105
1.55 47.2026658307
2.0 47.133650041
Smallest error 47.134 at a = 2.000
y5
0.2 45.8541548975
0.65 45.8173109884
1.1 45.7664630569
1.55 45.6702743607
2.0 45.5796259981
Smallest error 45.580 at a = 2.000
y6
0.2 44.8009826345
0.65 44.7931117899
1.1 44.7158633964
1.55 44.6110548119
2.0 44.506692815
Smallest error 44.507 at a = 2.000


In [33]:
# now there are three regressors. WE have to use them.
np.save('bestModels.npy',bestModels)

In [34]:
bestModels = np.load('bestModels.npy').item()

In [35]:
testSet.columns.values
dropCols = ['index','y','y1','y2','y3','y4','y5','y6']
tripVal = testSet.get(dropCols)
testSet.drop(dropCols,inplace = True, axis = 1)

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [37]:
def getTrips(X):
    model1 = bestModels.get('y1')
    model2 = bestModels.get('y2')
    model3 = bestModels.get('y3')
    model4 = bestModels.get('y4')
    model5 = bestModels.get('y5')
    model6 = bestModels.get('y6')
    pr1 = model1.predict(X)
    pr2 = model2.predict(X)
    pr3 = model3.predict(X)
    pr4 = model4.predict(X)
    pr5 = model5.predict(X)
    pr6 = model6.predict(X)
    pr1[pr1<0] = 0
    pr2[pr2<0] = 0
    pr3[pr3<0] = 0
    pr4[pr4<0] = 0
    pr5[pr5<0] = 0
    pr6[pr6<0] = 0
    
    return np.array([pr1, pr2, pr3, pr4, pr5, pr6])

In [48]:
prediction = getTrips(testSet)

predictionDf = pd.DataFrame(prediction.T,columns=['y1','y2','y3','y4','y5','y6'])
predictionDf.set_index(testSet.index,inplace=True)
predictionDf = predictionDf.merge(regDf,left_index=True,right_index=True,how='left')

# теперь надо сохранить это в файл
fName = 'res_week5-3.csv'

rnd = np.round

f = open(fName,'w')
f.writelines('id,y\n')

for ind, row in predictionDf.iterrows():
    historyStart = row.date - datetime.timedelta(hours = 1)
    
    if historyStart > datetime.datetime(2016,6,30,17):
        continue
    
    s0 = str(row.region)+'_'+ str(datetime.datetime.strftime(historyStart, "%Y-%m-%d"))+ '_'+ str(historyStart.hour)
    
    s1 = s0 +'_1,'+str(rnd(row.get('y1'))) + '\n'
    f.writelines(s1)

    s2 = s0 +'_2,'+str(rnd(row.get('y2'))) + '\n'
    f.writelines(s2)
    
    s3 = s0 +'_3,'+str(rnd(row.get('y3'))) + '\n'
    f.writelines(s3)

    s4 = s0 +'_4,'+str(rnd(row.get('y4'))) + '\n'
    f.writelines(s4)

    s5 = s0 +'_5,'+str(rnd(row.get('y5'))) + '\n'
    f.writelines(s5)

    s6 = s0 +'_6,'+str(rnd(row.get('y6'))) + '\n'
    f.writelines(s6)
    
f.close()    